# **Estruturas criptograficas: TP2 problema 1**

In [139]:
from sage.all import *
import os
import sys
import hashlib

In [147]:
#A point on (twisted) Edwards curve.
class EdwardsPoint:
    #base_field = None
    #x = None
    #y = None
    #z = None
    def initpoint(self, x, y):
        self.x=x
        self.y=y
        self.z=self.base_field.make(1)
    def decode_base(self,s,b):
        #Check that point encoding is the correct length.
        if len(s)!=b//8: return (None,None)
        #Extract signbit.
        xs=s[(b-1)//8]>>((b-1)&7)
        #Decode y.  If this fails, fail.
        y = self.base_field.frombytes(s,b)
        if y is None: return (None,None)
        #Try to recover x.  If it does not exist, or if zero and xs
        #are wrong, fail.
        x=self.solve_x2(y).sqrt()
        if x is None or (x.iszero() and xs!=x.sign()):
            return (None,None)
        #If sign of x isn't correct, flip it.
        if x.sign()!=xs: x=-x
        # Return the constructed point.
        return (x,y)
    def encode_base(self,b):
        xp,yp=self.x/self.z,self.y/self.z
        #Encode y.
        s=bytearray(yp.tobytes(b))
        #Add sign bit of x to encoding.
        if xp.sign()!=0: s[(b-1)//8]|=1<<(b-1)%8
    def __mul__(self,x):
        r=self.zero_elem()
        s=self
        while x > 0:
            if (x%2)>0:
                r=r+s
            s=s.double()
            x=x//2
        return r
    #Check that two points are equal.
    def __eq__(self,y):
        #Need to check x1/z1 == x2/z2 and similarly for y, so cross
        #multiply to eliminate divisions.
        xn1=self.x*y.z
        xn2=y.x*self.z
        yn1=self.y*y.z
        yn2=y.y*self.z
        return xn1==xn2 and yn1==yn2
    #Check if two points are not equal.
    def __ne__(self,y):
        return not (self==y)

In [150]:
def hexi(s): return int.from_bytes(bytes.fromhex(s),byteorder="big")

#A point on Edwards25519.
class Edwards25519Point(EdwardsPoint):
    #Create a new point on the curve.
    base_field=Field(1,2**255-19)
    d=-base_field.make(121665)/base_field.make(121666)
    f0=base_field.make(0)
    f1=base_field.make(1)
    xb=base_field.make(hexi("216936D3CD6E53FEC0A4E231FDD6DC5C692CC76"+\
        "09525A7B2C9562D608F25D51A"))
    yb=base_field.make(hexi("666666666666666666666666666666666666666"+\
        "6666666666666666666666658"))
    #The standard base point.
    @staticmethod
    def stdbase():
        return Edwards25519Point(Edwards25519Point.xb,\
            Edwards25519Point.yb)
    def __init__(self,x,y):
        #Check the point is actually on the curve.
        if y*y-x*x!=self.f1+self.d*x*x*y*y:
            raise ValueError("Invalid point")
        self.initpoint(x, y)
        self.t=x*y
    #Decode a point representation.
    def decode(self,s):
        x,y=self.decode_base(s,256);
    #Encode a point representation.
    def encode(self):
        return self.encode_base(256)
    #Construct a neutral point on this curve.
    def zero_elem(self):
        return Edwards25519Point(self.f0,self.f1)
    #Solve for x^2.
    def solve_x2(self,y):
        return ((y*y-self.f1)/(self.d*y*y+self.f1))
    #Point addition.
    def __add__(self,y):
        #The formulas are from EFD.
        tmp=self.zero_elem()
        zcp=self.z*y.z
        A=(self.y-self.x)*(y.y-y.x)
        B=(self.y+self.x)*(y.y+y.x)
        C=(self.d+self.d)*self.t*y.t
        D=zcp+zcp
        E,H=B-A,B+A
        F,G=D-C,D+C
        tmp.x,tmp.y,tmp.z,tmp.t=E*F,G*H,F*G,E*H
        return tmp
    #Point doubling.
    def double(self):
        #The formulas are from EFD (with assumption a=-1 propagated).
        tmp=self.zero_elem()
        A=self.x*self.x
        B=self.y*self.y
        Ch=self.z*self.z
        C=Ch+Ch
        H=A+B
        xys=self.x+self.y
        E=H-xys*xys
        G=A-B
        F=C+G
        tmp.x,tmp.y,tmp.z,tmp.t=E*F,G*H,F*G,E*H
        return tmp
    #Order of basepoint.
    def l(self):
        return hexi("1000000000000000000000000000000014def9dea2f79cd"+\
            "65812631a5cf5d3ed")
    #The logarithm of cofactor.
    def c(self): return 3
    #The highest set bit
    def n(self): return 254
    #The coding length
    def b(self): return 256
    #Validity check (for debugging)
    def is_valid_point(self):
        x,y,z,t=self.x,self.y,self.z,self.t
        x2=x*x
        y2=y*y
        z2=z*z
        lhs=(y2-x2)*z2
        rhs=z2*z2+self.d*x2*y2
        assert(lhs == rhs)
        assert(t*z == x*y)

TypeError: base ring 1 is no commutative ring

In [161]:
# Curva Edwards25519
class EdDSA:
    
    def __init__(self):
        self.b = 256
        self.p = 2^255 - 19
        self.ec = GF(self.p)
        self.x = self.ec(15112221349535400772501151409588531511454012693041857206046113283949847762202)
        self.y = self.ec(46316835694926478169428394003475163141307993866256225615783033603165251855960)
        self.c = 3
        self.n = 254
        self.d = 37095705934669439343138083508754565189542113879843219016388785533085940283555
        self.a = self.ec(-1)
        self.L = 2^252 + 27742317777372353535851937790883648493
    
        
    def H(self, k):
        return hashlib.sha512(k).digest()    
            
    def bytes_to_bits_binary(self, byte_data):
        bits_data = bin(int.from_bytes(byte_data, byteorder='big'))[2:]
        return bits_data

    def bit(self, h,i):
        return ((h[int(i/8)]) >> (i%8)) & 1 
    
    def edwards(self, P,Q):
        x1 = P[0]
        y1 = P[1]
        x2 = Q[0]
        y2 = Q[1]
        x3 = (x1*y2+x2*y1) * self.inv(1+self.d*x1*x2*y1*y2)
        y3 = (y1*y2+x1*x2) * self.inv(1-self.d*x1*x2*y1*y2)
        return [x3 % self.p,y3 % self.p]

    def scalarmult(self, P,e):
        if e == 0: return [0,1]
        Q = self.scalarmult(P,e/2)
        Q = self.edwards(Q,Q)
        if e & 1: Q = self.edwards(Q,P)
        return Q
    
    def encodepoint(self, P):
        x = P[0]
        y = P[1]
        bits = [(y >> i) & 1 for i in range(self.b - 1)] + [x & 1]
        return ''.join([chr(sum([bits[i * 8 + j] << j for j in range(8)])) for i in range(self.b/8)])
    
    
    def keygen(self):
        private_key = os.urandom(self.b / 8)
        
        pk = self.H(private_key)
        pk = pk[:32]
        
        bits = int.from_bytes(pk)
    
        bits = [int(digit) for digit in list(ZZ(bits).binary())]
        
        extra = self.b - len(bits)
        for i in range(extra):
            bits.insert(0, 0)
        
        bits[0] = bits[1] = bits[2] = 0
        bits[self.b-2] = 1
        bits[self.b-1] = 0
        
        bits = "".join(map(str, bits))

        s = int(bits[::-1], 2)
        print(s)
        
        print(self.scalarmult((self.x, self.y), s))

        # pubK = s * self.B
        
        # print(pubK)
        
        # a = 2^(self.b-2) + sum(2^i * self.bit(pk,i) for i in range(3,self.b-2))

        # print(a)

        # point = self.scalarmult((self.x, self.y), a)
        # P = (self.x, self.y)

        # point = (self.x * a, self.y * a)
        # pubK = self.encodepoint(point)
        
        # return (pk, pubK)
        
    def encodeint(self, y):
        bits = [(y >> i) & 1 for i in range(self.b)]
        return ''.join([chr(sum([bits[i * 8 + j] << j for j in range(8)])) for i in range(self.b/8)])

    def encodepoint(self, P):
        x = P[0]
        y = P[1]
        bits = [(y >> i) & 1 for i in range(self.b - 1)] + [x & 1]
        return ''.join([chr(sum([bits[i * 8 + j] << j for j in range(8)])) for i in range(self.b/8)])
        
        
    def decodeint(self, s):
        return sum(2^i * self.bit(s,i) for i in range(0,self.b))

    def decodepoint(self, s):
        y = sum(2^i * self.bit(s,i) for i in range(0,self.b-1))
        x = self.xrecover(y)
        if x & 1 != self.bit(s,self.b-1): x = self.p-x
        P = [x,y]
        if not self.isoncurve(P): raise Exception("decoding point that is not on curve")
        return P  
        
    def isoncurve(self, P):
        x = P[0]
        y = P[1]
        return (-x*x + y*y - 1 - self.d*x*x*y*y) % self.p == 0   
        
    def xrecover(self, y):
        I = self.expmod(2,(self.p-1)/4,self.p)
        
        xx = (y*y-1) * self.inv(self.d*y*y+1)
        x = self.expmod(xx,(self.p+3)/8,self.p)
        if (x*x - xx) % self.p != 0: x = (x*I) % self.p
        if x % 2 != 0: x = self.p-x
        return x   
    
    def expmod(self, b,e,m):
        if e == 0: return 1
        t = self.expmod(b,e/2,m)^2 % m
        if e & 1: t = (t*b) % m
        return t
    
    def inv(self, x):
        return self.expmod(x,self.p-2,self.p)

In [162]:
ed = EdDSA()
print(ed.keygen())

53278050650069118395020481574315371620011973875320722716271908042478898414664


RecursionError: maximum recursion depth exceeded in comparison

In [26]:
i = 123456789987654321

enc = ed.encodeint(i)
# dec = ed.decodeint(enc)
print(len(enc.encode()))
print(enc)
# print(dec)

37
±úRàK¶                        
